In [ ]:
! pip3 install boto3 python-dotenv

In [1]:
import boto3
from dotenv import load_dotenv, find_dotenv
import os
import json
import textwrap as tw

In [2]:
load_dotenv(find_dotenv())

True

### AWS Credentials

In [3]:
# === AWS Configuration === #
COGNITO_REGION = os.getenv("COGNITO_REGION")
BEDROCK_REGION = os.getenv("BEDROCK_REGION")
MODEL_ID1 = os.getenv("MODEL_ID1")
MODEL_ID2 = os.getenv("MODEL_ID2")
IDENTITY_POOL_ID = os.getenv("IDENTITY_POOL_ID")
USER_POOL_ID = os.getenv("USER_POOL_ID")
APP_CLIENT_ID = os.getenv("APP_CLIENT_ID")
USERNAME = os.getenv("USERNAME")
PASSWORD = os.getenv("PASSWORD")

In [4]:
# === Helper: Get AWS Credentials === #
def get_credentials(username, password):
    idp_client = boto3.client("cognito-idp", region_name=COGNITO_REGION)
    response = idp_client.initiate_auth(
        AuthFlow="USER_PASSWORD_AUTH",
        AuthParameters={"USERNAME": username, "PASSWORD": password},
        ClientId=APP_CLIENT_ID,
    )
    id_token = response["AuthenticationResult"]["IdToken"]

    identity_client = boto3.client("cognito-identity", region_name=COGNITO_REGION)
    identity_response = identity_client.get_id(
        IdentityPoolId=IDENTITY_POOL_ID,
        Logins={f"cognito-idp.{COGNITO_REGION}.amazonaws.com/{USER_POOL_ID}": id_token},
    )

    creds_response = identity_client.get_credentials_for_identity(
        IdentityId=identity_response["IdentityId"],
        Logins={f"cognito-idp.{COGNITO_REGION}.amazonaws.com/{USER_POOL_ID}": id_token},
    )

    return creds_response["Credentials"]

### Experimentation

##### Test 1 - Simple Invocation

In [45]:
credentials = get_credentials(USERNAME, PASSWORD)

bedrock = boto3.client(
    "bedrock-runtime",
    region_name=BEDROCK_REGION,
    aws_access_key_id=credentials["AccessKeyId"],
    aws_secret_access_key=credentials["SecretKey"],
    aws_session_token=credentials["SessionToken"],
)

body = json.dumps({
    "max_tokens": 1024, # required
    "anthropic_version": "bedrock-2023-05-31",
    "messages": [{"role": "user", "content": "Hi"}]
})

response = bedrock.invoke_model(
    body=body,
    modelId=MODEL_ID1,
)

In [19]:
type(response)

dict

In [20]:
response

{'ResponseMetadata': {'RequestId': 'e8c40dae-541c-4d60-b390-17f01b9498fe',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 28 Oct 2025 11:35:56 GMT',
   'content-type': 'application/json',
   'content-length': '277',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'e8c40dae-541c-4d60-b390-17f01b9498fe',
   'x-amzn-bedrock-invocation-latency': '619',
   'x-amzn-bedrock-output-token-count': '12',
   'x-amzn-bedrock-input-token-count': '8'},
  'RetryAttempts': 0},
 'contentType': 'application/json',
 'body': <botocore.response.StreamingBody at 0x11636d6f0>}

In [21]:
response.get('body')

In [44]:
response.get('body').read() # can only be read once

b'{"id":"msg_bdrk_01S4idA2UEtWrFS3YW1ZJDGL","type":"message","role":"assistant","model":"claude-3-5-sonnet-20241022","content":[{"type":"text","text":"Hello! How can I help you today?"}],"stop_reason":"end_turn","stop_sequence":null,"usage":{"input_tokens":8,"output_tokens":12}}'

In [46]:
responde_body = json.loads(response.get('body').read())

In [23]:
type(responde_body)

dict

In [40]:
responde_body

{'id': 'msg_bdrk_01R41zujHHC2haUa6VFqhmAN',
 'type': 'message',
 'role': 'assistant',
 'model': 'claude-3-5-sonnet-20241022',
 'content': [{'type': 'text',
   'text': 'I\'ll help you submit assignments in Canvas. Here\'s a step-by-step guide:\n\n1. Log in to your Canvas account\n\n2. Navigate to your course\n\n3. Find the assignment in one of these ways:\n- Click on "Assignments" in the left-hand menu\n- Look for it on the course homepage\n- Check the course calendar\n\n4. Click on the assignment title\n\n5. Look for the "Submit Assignment" button (usually blue)\n\n6. Choose your submission type:\n- Text Entry: Type directly into the text box\n- File Upload: Click "Choose File" or drag and drop your file\n- Website URL: Paste a link\n- Media Recording: Record audio/video\n\n7. After uploading or entering your work, click "Submit Assignment"\n\n8. You should see a confirmation message\n\nIf you still can\'t find the submit button:\n- Check if the assignment is open (not past due)\n- Mak

In [47]:
print(responde_body.get('content')[0].get('text'))

Hello! How can I help you today?


##### Test 2 - Citations

In [28]:
credentials = get_credentials(USERNAME, PASSWORD)

bedrock = boto3.client(
    "bedrock-runtime",
    region_name=BEDROCK_REGION,
    aws_access_key_id=credentials["AccessKeyId"],
    aws_secret_access_key=credentials["SecretKey"],
    aws_session_token=credentials["SessionToken"],
)

body = json.dumps({
    "max_tokens": 1024, # required
    "anthropic_version": "bedrock-2023-05-31",
    "messages": [
      {
        "role": "user",
        "content": [
          {
            "type": "document",
            "source": {
              "type": "text",
              "media_type": "text/plain",
              "data": "The grass is green. The sky is blue."
            },
            "title": "My Document",
            "context": "This is a trustworthy document.",
            "citations": {"enabled": True}
          },
          {
            "type": "text",
            "text": "What color is the grass and sky?"
          }
        ]
      }
    ]
})

response = bedrock.invoke_model(
    body=body,
    modelId=MODEL_ID1,
)

In [29]:
response

{'ResponseMetadata': {'RequestId': '6da56061-30be-46ab-bf3a-b669b1414129',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 28 Oct 2025 11:37:01 GMT',
   'content-type': 'application/json',
   'content-length': '751',
   'connection': 'keep-alive',
   'x-amzn-requestid': '6da56061-30be-46ab-bf3a-b669b1414129',
   'x-amzn-bedrock-invocation-latency': '1552',
   'x-amzn-bedrock-output-token-count': '52',
   'x-amzn-bedrock-input-token-count': '610'},
  'RetryAttempts': 0},
 'contentType': 'application/json',
 'body': <botocore.response.StreamingBody at 0x116406470>}

In [30]:
response_body = json.loads(response.get('body').read())

In [31]:
response_body

{'id': 'msg_bdrk_01Jo35ypxxWZzyXyL4jG2HUG',
 'type': 'message',
 'role': 'assistant',
 'model': 'claude-3-5-sonnet-20241022',
 'content': [{'type': 'text', 'text': 'Based on the provided document, '},
  {'citations': [{'type': 'char_location',
     'cited_text': 'The grass is green. ',
     'document_index': 0,
     'document_title': 'My Document',
     'start_char_index': 0,
     'end_char_index': 20}],
   'type': 'text',
   'text': 'the grass is green'},
  {'type': 'text', 'text': ' and '},
  {'citations': [{'type': 'char_location',
     'cited_text': 'The sky is blue.',
     'document_index': 0,
     'document_title': 'My Document',
     'start_char_index': 20,
     'end_char_index': 36}],
   'type': 'text',
   'text': 'the sky is blue'},
  {'type': 'text', 'text': '.'}],
 'stop_reason': 'end_turn',
 'stop_sequence': None,
 'usage': {'input_tokens': 610, 'output_tokens': 52}}

In [32]:
for block in response_body.get('content'):
    print(block.get('text'),end='')

Based on the provided document, the grass is green and the sky is blue.

In [33]:
# Citations not working with Claude 3 haiku

##### Test 3 - Simple Multi-Turn Chatbot

In [34]:
conversation_history = []

credentials = get_credentials(USERNAME, PASSWORD)

bedrock = boto3.client(
    "bedrock-runtime",
    region_name=BEDROCK_REGION,
    aws_access_key_id=credentials["AccessKeyId"],
    aws_secret_access_key=credentials["SecretKey"],
    aws_session_token=credentials["SessionToken"],
)

while True:
    user_input = input("User: ")

    if user_input.lower() in ("quit", "thank you", "bye"):
        print("---- END OF CONVERSATION -----")
        break

    conversation_history.append({
        "role": "user",
        "content": user_input
    })

    body = json.dumps({
        "max_tokens": 1000, # required
        "anthropic_version": "bedrock-2023-05-31",
        "messages": conversation_history
    })

    response = bedrock.invoke_model(
        body=body,
        modelId=MODEL_ID2,
    )

    response_body = json.loads(response.get('body').read())
    assistant_response = response_body.get('content')[0].get('text')

    conversation_history.append({
        "role":"assistant",
        "content": assistant_response
    })

    print(f"Assistant: {assistant_response}")

Assistant: Hello! How can I assist you today?
Assistant: I don't have a specific knowledge cut-off date that I'm aware of. My knowledge comes from the training data I was exposed to during the machine learning process that created me, but I'm not sure of the exact timeframe or cutoff for that data. Since I'm an AI assistant created by Anthropic, the best thing would be to check their website or reach out to them directly if you need more information about the specifics of my knowledge base and capabilities.
Assistant: I do not actually have an internal calendar or knowledge of the current date. As an AI system, I don't have a persistent sense of the current date or time. I can only respond based on the information provided to me in our conversation. If you share the current date with me, I'd be happy to acknowledge it, but I don't have an independent way to know the date on my own.
Assistant: I do not have any information about the 2025 AFL grand final winner, as that is in the future 

##### Test 4 - `stop_sequences`

In [37]:
credentials = get_credentials(USERNAME, PASSWORD)

bedrock = boto3.client(
    "bedrock-runtime",
    region_name=BEDROCK_REGION,
    aws_access_key_id=credentials["AccessKeyId"],
    aws_secret_access_key=credentials["SecretKey"],
    aws_session_token=credentials["SessionToken"],
)

body = json.dumps({
    "max_tokens": 1024, # required
    "stop_sequences": ['a','h','b'],
    "anthropic_version": "bedrock-2023-05-31",
    "messages": [{"role": "user", "content": "Hi"}]
})

response = bedrock.invoke_model(
    body=body,
    modelId=MODEL_ID1,
)

In [38]:
response_body = json.loads(response.get('body').read())

In [39]:
response_body

{'id': 'msg_bdrk_01HhxHkAXmFoJbmzfTX8AzQW',
 'type': 'message',
 'role': 'assistant',
 'model': 'claude-3-5-sonnet-20241022',
 'content': [{'type': 'text', 'text': 'Hello! How c'}],
 'stop_reason': 'stop_sequence',
 'stop_sequence': 'a',
 'usage': {'input_tokens': 8, 'output_tokens': 4}}

In [42]:
print(response_body.get('content')[0].get('text'))

Hello! How c


In [43]:
print(response_body.get('stop_reason'))

stop_sequence


In [44]:
print(response_body.get('stop_sequence'))

a


##### Test 5 - `temperature` + few-shot prompting

The temperature parameter is used to control the "randomness" and "creativity" of the generated responses. It ranges from 0 to 1, with higher values resulting in more diverse and unpredictable responses with variations in phrasing. Lower temperatures can result in more deterministic outputs that stick to the most probable phrasing and answers. Temperature has a default value of 1.

In [55]:
task = """Come up with a unique soccer team name.
(IMPORTANT) Respond with a JSON object containing a single key "team_name".

Example Output: '{"team_name": "<output>"}'"""

for _ in range(10):
    credentials = get_credentials(USERNAME, PASSWORD)

    bedrock = boto3.client(
        "bedrock-runtime",
        region_name=BEDROCK_REGION,
        aws_access_key_id=credentials["AccessKeyId"],
        aws_secret_access_key=credentials["SecretKey"],
        aws_session_token=credentials["SessionToken"],
    )

    body = json.dumps({
        "max_tokens": 1024, # required
        "temperature": 1,
        "anthropic_version": "bedrock-2023-05-31",
        "messages": [
            {"role": "user", "content": task},
            {"role": "assistant", "content": '{"team_name":"Liverpool"}'},
            {"role": "user", "content": task},
            {"role": "assistant", "content": '{"team_name":"Real Madrid CF"}'},
            {"role": "user", "content": task},
            {"role": "assistant", "content": '{"team_name":"SSC Napoli"}'},
            {"role": "user", "content": task},
        ]
    })

    response = bedrock.invoke_model(
        body=body,
        modelId=MODEL_ID1,
    )

    ai_reply = json.loads(json.loads(response.get('body').read()).get('content')[0].get('text')).get('team_name')
    print(ai_reply)

Ajax Amsterdam
Athletico Dynamo
Borussia Dortmund
Bayern Munich
Bayern Munich
Olympique Lyonnais
Atletico Madrid
Bayern Munich
Bayern Munich
Borussia Dortmund


##### Test 6 - System Prompt

In [9]:
import textwrap

In [10]:
pp = textwrap.TextWrapper(width=100)

In [11]:
conversation_history = [{"role":"user", "content":"👋"}]

credentials = get_credentials(USERNAME, PASSWORD)

bedrock = boto3.client(
    "bedrock-runtime",
    region_name=BEDROCK_REGION,
    aws_access_key_id=credentials["AccessKeyId"],
    aws_secret_access_key=credentials["SecretKey"],
    aws_session_token=credentials["SessionToken"],
)

system = """"You are "ARTIM", an AI Support Chatbot designed to assist student with queries/ troubleshooting related to Canvas Learning Management System.
(IMPORTANT) You are to always conduct yourself in a professional and ethical manner regardless of user's actions/ words.
(IMPORTANT) Refrain from repling to off-topic questions, instead reply with a polite message stating your intended role/ responsibilities.
(IMPORTANT) Strictly REFRAIN from generating harmful content
(IMPORTANT) If you have insufficient knowledge to assist with the user's query, do your best and escalate the query to "IT SUPPORT"

Please introduce yourself at the start of any conversation."""

START=0
while True:
    if START:
        user_input = input("You: ")
        print(pp.fill(f"You: {user_input}"))
        print()

        if user_input.lower() in ("quit", "thank you", "bye"):
            print("---- END OF CONVERSATION -----")
            break

        conversation_history.append({
            "role": "user",
            "content": user_input
        })
    
    START=1

    body = json.dumps({
        "max_tokens": 2000, # required
        "system": system,
        "temperature": 0.2,
        "anthropic_version": "bedrock-2023-05-31",
        "messages": conversation_history
    })

    response = bedrock.invoke_model(
        body=body,
        modelId=MODEL_ID2,
    )

    response_body = json.loads(response.get('body').read())
    assistant_response = response_body.get('content')[0].get('text')

    conversation_history.append({
        "role":"assistant",
        "content": assistant_response
    })

    print(pp.fill(f"Assistant: {assistant_response}"))
    print()

Assistant: Hello! I'm ARTIM, an AI assistant created to help students with questions and
troubleshooting related to the Canvas learning management system. How can I assist you today?

You: I am a new student and my course starts tomorrow. It is an online course. I have logged into
the student portal with my personal email and it says that I am enrolled in the course, but when I
log into canvas with my new TAFE email it says I am not enrolled in any courses.  It is also a
Sunday so I cannot call up for immediate help.  What do I do given the course is supposed to start
tomorrow?

Assistant: I understand your concern as a new student starting an online course tomorrow. Let me try
to assist you with this issue.  Since you are able to see your enrollment in the student portal
using your personal email, that's a good sign. However, the discrepancy in your enrollment status on
Canvas is concerning.  Here are a few suggestions on what you can do:  1. Double-check that you are
logging into Can

In [ ]:
! pip3 install ipywidgets

In [6]:
import ipywidgets as widgets
from IPython.display import display, Markdown, HTML

user = widgets.Output(layout=widgets.Layout(width='50%', border='5px solid orange', white_space='normal', overflow='auto'))
with user:
    display(Markdown("# USER"))
    display(Markdown("<br>"))

assistant = widgets.Output(layout=widgets.Layout(width='50%', border='5px solid pink', white_space='normal', overflow='auto'))
with assistant:
    display(Markdown("# ARTIM"))
    display(Markdown("<br>"))

# Arrange the output widgets horizontally
two_columns = widgets.HBox([assistant, user])

# Display the two-column layout
display(two_columns)

conversation_history = [{"role":"user", "content":"👋"}]

credentials = get_credentials(USERNAME, PASSWORD)

bedrock = boto3.client(
    "bedrock-runtime",
    region_name=BEDROCK_REGION,
    aws_access_key_id=credentials["AccessKeyId"],
    aws_secret_access_key=credentials["SecretKey"],
    aws_session_token=credentials["SessionToken"],
)

system = """"You are "ARTIM", an AI Support Chatbot designed to assist student with queries/ troubleshooting related to Canvas Learning Management System.
(IMPORTANT) You are to always conduct yourself in a professional and ethical manner regardless of user's actions/ words.
(IMPORTANT) Refrain from repling to off-topic questions, instead reply with a polite message stating your intended role/ responsibilities.
(IMPORTANT) Strictly REFRAIN from generating harmful content
(IMPORTANT) If you have insufficient knowledge to assist with the user's query, do your best and escalate the query to "IT SUPPORT"

Please introduce yourself at the start of any conversation."""


START=0
while True:
    if START:
        user_input = input("You: ")
        with user:
            display(HTML("<div style='white-space: normal; word-wrap: break-word; font-size: 22px;'>"
            f"<span style='font-weight:bold'>You:</span> {user_input}"
            "</span>"))
            display(Markdown("<br>"))

        if user_input.lower() in ("quit", "thank you", "bye"):
            with assistant:
                display(HTML("<div style='white-space: normal; word-wrap: break-word; font-size: 22px;'>"
                "---- END OF CONVERSATION -----"
                "</div>"))
                display(Markdown("<br>"))
            break

        conversation_history.append({
            "role": "user",
            "content": user_input
        })
    
    START=1

    body = json.dumps({
        "max_tokens": 2000, # required
        "system": system,
        "temperature": 0.2,
        "anthropic_version": "bedrock-2023-05-31",
        "messages": conversation_history
    })

    response = bedrock.invoke_model(
        body=body,
        modelId=MODEL_ID2,
    )

    response_body = json.loads(response.get('body').read())
    assistant_response = response_body.get('content')[0].get('text')

    conversation_history.append({
        "role":"assistant",
        "content": assistant_response
    })

    with assistant:
        display(HTML("<div style='white-space: normal; word-wrap: break-word; font-size: 22px;'>"
        f"<span style='font-weight:bold'>ARTIM:</span> {assistant_response}"
        "</div>"))
        display(Markdown("<br>"))

##### Test 7 - Streaming

In [16]:
credentials = get_credentials(USERNAME, PASSWORD)

bedrock = boto3.client(
    "bedrock-runtime",
    region_name=BEDROCK_REGION,
    aws_access_key_id=credentials["AccessKeyId"],
    aws_secret_access_key=credentials["SecretKey"],
    aws_session_token=credentials["SessionToken"],
)

body = json.dumps({
    "max_tokens": 1024, # required
    "temperature": 0,
    "anthropic_version": "bedrock-2023-05-31",
    "messages": [{"role": "user", "content": "Write a short 200 word summary about Model Context Protocol"}]
})

stream = bedrock.invoke_model_with_response_stream(
    body=body,
    modelId=MODEL_ID1,
)

In [17]:
stream

{'ResponseMetadata': {'RequestId': '35e1f59a-4e8f-4339-825b-50bb635b3ed8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 31 Oct 2025 03:48:17 GMT',
   'content-type': 'application/vnd.amazon.eventstream',
   'transfer-encoding': 'chunked',
   'connection': 'keep-alive',
   'x-amzn-requestid': '35e1f59a-4e8f-4339-825b-50bb635b3ed8',
   'x-amzn-bedrock-content-type': 'application/json'},
  'RetryAttempts': 0},
 'contentType': 'application/json',
 'body': <botocore.eventstream.EventStream at 0x10d148d10>}

In [18]:
for event in stream.get('body'):
    print(event)

{'chunk': {'bytes': b'{"type":"message_start","message":{"id":"msg_bdrk_01AP2KBHS6kwunKwruQcuicj","type":"message","role":"assistant","model":"claude-3-5-sonnet-20241022","content":[],"stop_reason":null,"stop_sequence":null,"usage":{"input_tokens":19,"output_tokens":1}}}'}}
{'chunk': {'bytes': b'{"type":"content_block_start","index":0,"content_block":{"type":"text","text":""}}'}}
{'chunk': {'bytes': b'{"type":"content_block_delta","index":0,"delta":{"type":"text_delta","text":"Model"}}'}}
{'chunk': {'bytes': b'{"type":"content_block_delta","index":0,"delta":{"type":"text_delta","text":" Context Protocol (M"}}'}}
{'chunk': {'bytes': b'{"type":"content_block_delta","index":0,"delta":{"type":"text_delta","text":"CP) is a"}}'}}
{'chunk': {'bytes': b'{"type":"content_block_delta","index":0,"delta":{"type":"text_delta","text":" framework"}}'}}
{'chunk': {'bytes': b'{"type":"content_block_delta","index":0,"delta":{"type":"text_delta","text":" designe"}}'}}
{'chunk': {'bytes': b'{"type":"conte

In [50]:
credentials = get_credentials(USERNAME, PASSWORD)

bedrock = boto3.client(
    "bedrock-runtime",
    region_name=BEDROCK_REGION,
    aws_access_key_id=credentials["AccessKeyId"],
    aws_secret_access_key=credentials["SecretKey"],
    aws_session_token=credentials["SessionToken"],
)

body = json.dumps({
    "max_tokens": 2500, # required
    "temperature": 0,
    "anthropic_version": "bedrock-2023-05-31",
    "messages": [{"role": "user", "content": "Explain about Model Context Protocol in an intuitive way"}]
})

stream = bedrock.invoke_model_with_response_stream(
    body=body,
    modelId=MODEL_ID1,
)

for event in stream.get('body'):
    chunk = event.get('chunk')
    sse = json.loads(chunk.get('bytes'))

    if sse.get('type') == "content_block_delta":
        print(sse.get('delta').get('text'), end='', flush=True)


Let me explain Model Context Protocol (MCP) in a simple and intuitive way.

Think of MCP like a "user manual" for AI models that helps them understand:
1. What they can and cannot do
2. How they should behave
3. What their purpose is

Here's an analogy to help understand it better:

Imagine you're hiring a new employee 👩‍💼

1. Job Description:
- Without MCP: The employee has vague ideas about their role
- With MCP: They have clear documentation about their responsibilities

2. Company Guidelines:
- Without MCP: Employee might act inconsistently
- With MCP: Clear rules about professional conduct

Key Components of MCP:

1. Model Identity 🎭
- Who am I?
- What's my purpose?
- What are my capabilities?

2. Behavioral Guidelines 📋
- How should I respond?
- What tone should I use?
- What are my limitations?

3. Task Context 🎯
- What am I trying to achieve?
- Who am I helping?
- What's the desired outcome?

Real-world example:
```
MCP for a Customer Service AI:

Identity:
- I am a helpful cus

##### 8 - Tool use

###### **Experiment 1**

In [7]:
credentials = get_credentials(USERNAME, PASSWORD)

In [8]:
bedrock = boto3.client(
    "bedrock-runtime",
    region_name=BEDROCK_REGION,
    aws_access_key_id=credentials["AccessKeyId"],
    aws_secret_access_key=credentials["SecretKey"],
    aws_session_token=credentials["SessionToken"],
)

body = json.dumps({
    "max_tokens": 1024, # required
    "temperature": 0,
    "anthropic_version": "bedrock-2023-05-31",
    "tools": [
      {
        "name": "fetch_canvas_guides",
        "description": "Based on user query, fetch related information from canvas guides",
        "input_schema": {
          "type": "object",
          "properties": {
              "user_query": {
                  "type": "string",
                  "description": "User query string *AS IS* to search related documents"
              }
          },
          "required": ["user_query"]
        }
      }
    ],
    "system": """You are "ARTIM", an AI Support Chatbot designed to assist student with queries/ troubleshooting related to Canvas Learning Management System. You are to always conduct yourself in a professional and ethical manner regardless of user's actions/ words. Refrain from repling to off-topic questions, instead reply with a polite message stating your intended role/ responsibilities. Strictly REFRAIN from generating harmful content. If you have insufficient knowledge to assist with the user's query, consult 'fetch_canvas_guides' tool. 
    (VERY IMPORTANT) When creating a tool invocation for 'fetch_canvas_guides', never modify the original user message. Pass the exact raw user text to the tool under "user_query".
    If you still find it hard to assist with user query despite consulting 'fetch_canvas_guides' tool, escalate the query to "IT SUPPORT.""",
    "messages": [{"role": "user", "content": "I am unable to submit my assignment! I need help"}]
})



response = bedrock.invoke_model(
    body=body,
    modelId=MODEL_ID1,
)

assistant_response = json.loads(response.get('body').read())

In [9]:
assistant_response

{'id': 'msg_bdrk_01PN4ZNYoHdECYL9LQmFavJN',
 'type': 'message',
 'role': 'assistant',
 'model': 'claude-3-5-sonnet-20241022',
 'content': [{'type': 'text',
   'text': 'Let me help you troubleshoot your assignment submission issue. Let me search for relevant information from Canvas guides to better assist you.'},
  {'type': 'tool_use',
   'id': 'toolu_bdrk_01F2md9RZnySwkNwWxNizAv6',
   'name': 'fetch_canvas_guides',
   'input': {'user_query': 'I am unable to submit my assignment! I need help'}}],
 'stop_reason': 'tool_use',
 'stop_sequence': None,
 'usage': {'input_tokens': 606, 'output_tokens': 94}}

In [10]:
conversation_history = [
    {"role": "user", "content": "I am unable to submit my assignment! I need help"},
    {"role": "assistant", "content":[
        {'type': 'text',
         'text': 'Let me help you troubleshoot your assignment submission issue. Let me check the Canvas guides for relevant information about assignment submission problems.'
        },
        {'type': 'tool_use',
        'id': 'toolu_bdrk_01KsKYsm95xnVKua9DTyKvDR',
        'name': 'fetch_canvas_guides',
        'input': {'user_query': 'I am unable to submit my assignment! I need help'}
        }]
    },
    {"role":"user", "content": [
        {"type": "tool_result",
         'tool_use_id': 'toolu_bdrk_01KsKYsm95xnVKua9DTyKvDR',
         "content": """How do I submit an online assignment?You can submit online assignments in Canvas using several submission types. Instructors can choose what kind of online submissions they want you to use. You may also have the option to resubmit assignments if your instructor allows.Files uploaded using the Rich Content Editor count toward your user storage quota. Any attachments added as part of a graded assignment submission are also copied to your user files but are not counted against your user quota. However, once the file has been uploaded as a submission, you cannot delete the file. Submitted files are stored in the Submissions folder.Before submitting an assignment, you may want to review all assignment information, such as the assignment rubric, if any.This lesson shows how to turn in a standard online assignment. Learn how to submit a peer review assignment.Third-Party File Application SubmissionsYou can submit assignments from Google Drive, Dropbox, or another third-party service via your desktop computer in one of two ways:Download the file to your computer and submit as a File UploadShare the file, copy the file URL, and submit as a Website URLMobile SubmissionsYou can also submit assignments using your Android or iOS device.Notes: Depending on the assignment submission type set by your instructor, not all file types may be available for your assignment submission.Not all of your assignments may be submitted online. If you cannot see the Start Assignment button, your instructor may want you to submit your assignment in a different way or the availability date has passed. View the description of the assignment for instructions, or contact your instructor for assistance.Canvas does not support file uploads larger than 5 GB.If enabled in your account, Canvas plays a celebration animation (confetti) when you submit an assignment on time. However, if you prefer, you can disable this feature option in your user settings.If the assignment you are accessing displays differently, your assignment may be using the Assignment Enhancements feature.Group assignments do not support the student annotation submission type.If the assignment uses file restrictions, submitting through the Google Drive tab results in an error. In this case, download the file from Google Drive and submit it using the file upload tab instead. Open AssignmentsIn Course Navigation, click the Assignments link.Note: You can also access your Assignments through your user or course dashboard, the Syllabus, Gradebook, Calendar, or Modules.View Course AssignmentsClick the name of an assignment.Start AssignmentTo submit an assignment, click the Start Assignment button.Note: If you cannot see the Start Assignment button, your instructor may not want you to submit your assignment online or the availability date has passed. View the description of the assignment for instructions, or contact your instructor for assistance.View Submission AttemptsYour instructor may limit the number of submission attempts you are allotted for an assignment. If your assignment has a limited number of submissions, you can view the number of submission attempts you have made [1] and the number of submissions allowed for the assignment [2].Once you have used all your submission attempts, the New Attempt button displays as disabled [3].View Final Grade NoticeA banner may appear above your assignment to indicate that your instructor has removed the assignment from total grade calculations. However, this setting does not affect assignment submissions.Select Submission TypeYour instructor will decide what kinds of submissions are appropriate for each Assignment. There are four submission types: upload a file, submit a text entry, enter a website URL, or submit media. You can only select one submission type per submission.Note: Depending on the assignment submission type set by your instructor, not all file types may be available for your assignment submission.Submit Student AnnotationTo submit an annotated document, click the Student Annotation tab.Note: Group assignments do not support the student annotation submission type.Submit a File UploadTo upload a file from your computer or take a photo using your webcam and submit it as your assignment, select the File Upload tab.Submit a Text EntryTo submit a text entry assignment, select the Text Entry tab. Note: The assignment submission Rich Content Editor includes a word count display below the bottom right corner of the text box.Submit Website URLTo submit a website URL, select the Website URL tab.Submit Media RecordingTo submit a media recording, select the Media tab.Submit AssignmentWhen you are ready to submit your assignment, click the Submit Assignment button. Note: Large files submitted using the File Upload tab display a submission status indicator.View SubmissionAfter you have submitted your work, you will see information in the Sidebar about your submission [1]. For file uploads, the sidebar provides a link to your submission to download if necessary.If you choose, you may resubmit another version of your assignment using the New Attempt button [2]. You can view the details of your most recent submission in the Sidebar, but your instructor can see all of your submissions. Once the instructor has graded your submission, the Grades link in Course Navigation displays a grading indicator.You can also see details about your assignment and links to additional feedback in your Grades page.Notes: Your assignment still displays in Assignments page and the Syllabus; the listing is not removed with assignment submissions.When you resubmit an assignment, you can only access and view your most recent submission. However, instructors can view all of your submissions.Self-Assessment SubmissionIf enabled by your instructor, you may be required to fill out a self-assessment rubric for your submission."""
        }]
    }
]

In [11]:
body = json.dumps({
    "max_tokens": 1024, # required
    "temperature": 0,
    "anthropic_version": "bedrock-2023-05-31",
    "tools": [
      {
        "name": "fetch_canvas_guides",
        "description": "Based on user query, fetch related information from canvas guides",
        "input_schema": {
          "type": "object",
          "properties": {
              "user_query": {
                  "type": "string",
                  "description": "User query string *AS IS* to search related documents"
              }
          },
          "required": ["user_query"]
        }
      }
    ],
    "system": """You are "ARTIM", an AI Support Chatbot designed to assist student with queries/ troubleshooting related to Canvas Learning Management System. You are to always conduct yourself in a professional and ethical manner regardless of user's actions/ words. Refrain from repling to off-topic questions, instead reply with a polite message stating your intended role/ responsibilities. Strictly REFRAIN from generating harmful content. If you have insufficient knowledge to assist with the user's query, consult 'fetch_canvas_guides' tool. 
    (VERY IMPORTANT) When creating a tool invocation for 'fetch_canvas_guides', never modify the original user message. Pass the exact raw user text to the tool under "user_query".
    If you still find it hard to assist with user query despite consulting 'fetch_canvas_guides' tool, escalate the query to "IT SUPPORT.""",
    "messages": conversation_history
})

response = bedrock.invoke_model(
    body=body,
    modelId=MODEL_ID1,
)

assistant_response = json.loads(response.get('body').read())

In [12]:
assistant_response

{'id': 'msg_bdrk_013sNpWJ47c1M92DCtTcGr2T',
 'type': 'message',
 'role': 'assistant',
 'model': 'claude-3-5-sonnet-20241022',
 'content': [{'type': 'text',
   'text': 'Based on the information from Canvas guides, let me help you troubleshoot your assignment submission issue. There could be several reasons why you\'re unable to submit your assignment. Let me walk you through some key points:\n\n1. First, check if you can see the "Start Assignment" button. If you can\'t see this button:\n   - The assignment might not be set up for online submission\n   - The assignment due date might have passed\n   - Your instructor might want you to submit it in a different way\n\n2. If you can see the "Start Assignment" button, verify the submission type allowed by your instructor:\n   - File Upload\n   - Text Entry\n   - Website URL\n   - Media Recording\n   - Student Annotation\n\n3. Important technical limitations to keep in mind:\n   - Canvas doesn\'t support file uploads larger than 5GB\n   - If 

In [13]:
print(assistant_response.get('content')[0].get('text'))

Based on the information from Canvas guides, let me help you troubleshoot your assignment submission issue. There could be several reasons why you're unable to submit your assignment. Let me walk you through some key points:

1. First, check if you can see the "Start Assignment" button. If you can't see this button:
   - The assignment might not be set up for online submission
   - The assignment due date might have passed
   - Your instructor might want you to submit it in a different way

2. If you can see the "Start Assignment" button, verify the submission type allowed by your instructor:
   - File Upload
   - Text Entry
   - Website URL
   - Media Recording
   - Student Annotation

3. Important technical limitations to keep in mind:
   - Canvas doesn't support file uploads larger than 5GB
   - If the assignment has file restrictions and you're trying to submit through Google Drive, you might need to download the file first and then upload it directly

To help you more specifically

###### **Experiment 2**

In [6]:
RED = "\033[31m"
GREEN = "\033[32m"
RESET = "\033[0m"

In [ ]:
conversation_history = [{"role":"user", "content":"👋"}]

credentials = get_credentials(USERNAME, PASSWORD)

bedrock = boto3.client(
    "bedrock-runtime",
    region_name=BEDROCK_REGION,
    aws_access_key_id=credentials["AccessKeyId"],
    aws_secret_access_key=credentials["SecretKey"],
    aws_session_token=credentials["SessionToken"],
)

system = """"You are "ARTIM", an AI Support Chatbot designed to assist student with queries/ troubleshooting related to Canvas Learning Management System.
(IMPORTANT) You are to always conduct yourself in a professional and ethical manner regardless of user's actions/ words.
(IMPORTANT) Refrain from repling to off-topic questions, instead reply with a polite message stating your intended role/ responsibilities.
(IMPORTANT) Strictly REFRAIN from generating harmful content
(IMPORTANT) If you have insufficient knowledge to assist with the user's query, always use 'fetch_canvas_guides' tool.
(VERY IMPORTANT) When creating a tool invocation for 'fetch_canvas_guides', NEVER modify the original user message. Pass the exact raw user text to the tool under "user_query".
(VERY IMPORTANT) Once you have obtained information from 'fetch_canvas_guides' tool. Your answer to the user message is to be STRICTLY grounded in the obtained information. Do NOT use your general knowledge BUT use your common sense.
(VERY IMPORTANT) If you still find it hard to assist with user query despite consulting 'fetch_canvas_guides' tool, escalate the query to "RMIT IT SUPPORT".

RMIT IT SUPPORT DETAILS:
Phone-support: +61399258000
Email-support: support@rmit.com
"""

USER=0
while True:
    if USER:
        user_input = input("You: ")

        conversation_history.append({
            "role": "user",
            "content": [{"type": "text", "text": user_input}]
        })

        print(pp.fill(f"You:\n{RED}{user_input}{RESET}"))
        print("-"*50)

        if user_input.lower() in ("quit", "thank you", "bye"):
            print("---- END OF CONVERSATION -----")
            break

       
    
    USER=1

    body = json.dumps({
        "max_tokens": 2000, # required
        "system": system,
        "tools": [
            {
                "name": "fetch_canvas_guides",
                "description": "fetch information from canvas guides",
                "input_schema": {
                "type": "object",
                "properties": {
                    "user_query": {
                        "type": "string",
                        "description": " raw user text/ message/ query to search related documents"
                    }
                },
                "required": ["user_query"]
                }
            }
        ],
        "temperature": 0.2,
        "anthropic_version": "bedrock-2023-05-31",
        "messages": conversation_history
    })

    response = bedrock.invoke_model(
        body=body,
        modelId=MODEL_ID1,
    )

    response_body = json.loads(response.get('body').read())
    

    conversation_history.append({
        "role":"assistant",
        "content": response_body.get('content')
    })

    if response_body.get('content')[-1].get('type') == 'tool_use':
        print("\033[34mUsing Tool...\033[0m")
        print("*"*50)
        fetched_canvas_info = input('Input Canvas Info: ')
        conversation_history.append({
            "role":"user",
            "content": [{
                'type': 'tool_result',
                'tool_use_id': response_body.get('content')[-1].get('id'),
                "content": fetched_canvas_info
            }]
        })
        
        USER=0
        continue


    assistant_response = response_body.get('content')[0].get('text')

    print(f"Assistant:\n{GREEN}{assistant_response}{RESET}")
    print("-"*50)

Assistant:
Hello! I'm ARTIM, an AI Support Chatbot dedicated to helping you with Canvas Learning Management System related queries. How can I assist you today with Canvas?
--------------------------------------------------
You: where to find the rubric for my assignment?
--------------------------------------------------
Using Tool...
**************************************************
Assistant:
Based on the information, here's how you can find the rubric for your assignment:

1. Go to Course Navigation and click on the "Assignments" link
2. Find and click on the name of your assignment
3. Scroll down below the assignment instructions - the rubric will be displayed there

The rubric will show you:
- Criteria: What you'll be evaluated on
- Ratings: Different levels of performance
- Point values: The points assigned to each rating (if your instructor has included them)

Important notes:
- Not all assignments include rubrics - it depends on whether your instructor has added one
- If your 

In [13]:
conversation_history

[{'role': 'user', 'content': '👋'},
 {'role': 'assistant',
  'content': [{'type': 'text',
    'text': "Hello! I'm ARTIM, an AI Support Chatbot dedicated to helping you with Canvas Learning Management System related queries. How can I assist you today with Canvas?"}]},
 {'role': 'user',
  'content': [{'type': 'text',
    'text': 'where to find the rubric for my assignment?'}]},
 {'role': 'assistant',
  'content': [{'type': 'text',
    'text': 'Let me help you find information about accessing assignment rubrics in Canvas.'},
   {'type': 'tool_use',
    'id': 'toolu_bdrk_01A7L8iseUF1ru8Q4dE5P4yA',
    'name': 'fetch_canvas_guides',
    'input': {'user_query': 'where to find the rubric for my assignment?'}}]},
 {'role': 'user',
  'content': [{'type': 'tool_result',
    'tool_use_id': 'toolu_bdrk_01A7L8iseUF1ru8Q4dE5P4yA',
    'content': "How do I view the rubric for my assignment?Your instructor may include a rubric as part of your assignment. The Rubric is a set of criteria that your instr

###### **Experiment 3**

In [7]:
RED = "\033[31m"
GREEN = "\033[32m"
BLUE = "\033[34m"
RESET = "\033[0m"

In [14]:
conversation_history = [{"role":"user", "content":"👋"}]

credentials = get_credentials(USERNAME, PASSWORD)

bedrock = boto3.client(
    "bedrock-runtime",
    region_name=BEDROCK_REGION,
    aws_access_key_id=credentials["AccessKeyId"],
    aws_secret_access_key=credentials["SecretKey"],
    aws_session_token=credentials["SessionToken"],
)

system = """"You are "ARTIM", an AI Support Chatbot at RMIT designed to assist student with queries/ troubleshooting related to Canvas Learning Management System.
(IMPORTANT) You are to always conduct yourself in a professional and ethical manner regardless of user's actions/ words.
(IMPORTANT) Refrain from repling to off-topic questions, instead reply with a polite message stating your intended role/ responsibilities.
(IMPORTANT) Strictly REFRAIN from generating harmful content
(IMPORTANT) If you have insufficient knowledge to assist with the user's query, always use 'fetch_canvas_guides' tool.
(VERY IMPORTANT) When creating a tool invocation for 'fetch_canvas_guides', NEVER modify the original user message. Pass the exact raw user text to the tool under "user_query".
(IMPORTANT) For Multi-hop queries, use parallel tool calls. Break down the user query into smaller sub-queries and address each sub-query individually using the relevant tools. After obtaining information for each sub-query, synthesize the information to provide a comprehensive response to the user's original query.
(VERY IMPORTANT) Once you have obtained information from 'fetch_canvas_guides' tool. Your answer to the user message is to be STRICTLY grounded in the obtained information. Do NOT use your general knowledge BUT use your common sense.
(VERY IMPORTANT) If you still find it hard to assist with user query despite consulting 'fetch_canvas_guides' tool, escalate the query to "RMIT IT SUPPORT".

RMIT IT SUPPORT DETAILS:
Phone-support: +61399258000
Email-support: support@rmit.com
"""

#For Multi-hop queries, try to break down the user query into smaller sub-queries (Parallel Tool Call) and address each #sub-query individually using the relevant tools. After obtaining information for each sub-query, synthesize the information #to provide a comprehensive response to the user's original query.


USER=0
while True:
    if USER:
        user_input = input("You: ")

        conversation_history.append({
            "role": "user",
            "content": [{"type": "text", "text": user_input +"\nPlease break down my query into smaller sub-queries if necessary, BUT maintain **exact words** for each sub-query from the query above. Address each sub-query individually using the relevant tools. After obtaining information for each sub-query, synthesize the information to provide a consice response to my original query."}]
        })

        print(pp.fill(f"You:\n{RED}{user_input}{RESET}"))
        print("-"*50)

        if user_input.lower() in ("quit", "thank you", "bye"):
            print("---- END OF CONVERSATION -----")
            break

       
    
    USER=1

    body = json.dumps({
        "max_tokens": 2500, # required
        "system": system,
        "tools": [
            {
                "name": "fetch_canvas_guides",
                "description": "fetch information from canvas guides",
                "input_schema": {
                "type": "object",
                "properties": {
                    "user_query": {
                        "type": "string",
                        "description": " One signle raw user text/ message/ query to search related documents"
                    }
                },
                "required": ["user_query"]
                }
            }
        ],
        "temperature": 0.2,
        "anthropic_version": "bedrock-2023-05-31",
        "messages": conversation_history
    })

    response = bedrock.invoke_model(
        body=body,
        modelId=MODEL_ID1,
    )

    response_body = json.loads(response.get('body').read())
    

    conversation_history.append({
        "role":"assistant",
        "content": response_body.get('content')
    })

    if response_body.get('content')[-1].get('type') == 'tool_use':
        print(f"{BLUE}Fetching Necessary Info...{RESET}")
        print("*"*50)

        tool_calls = []
        for content_block in response_body.get('content'):
            if content_block.get('type') == 'tool_use':
                tool_calls.append(content_block)

        tool_result_res = {
            "role":"user",
            "content": []
        }

        for tool_call in tool_calls:
            tool_result_res["content"].append({
                'type': 'tool_result',
                'tool_use_id': tool_call.get('id'),
                "content": input('Input Canvas Info: ')
            })
        conversation_history.append(tool_result_res)
        
        USER=0
        continue


    assistant_response = response_body.get('content')[0].get('text')

    print(f"Assistant:\n{GREEN}{assistant_response}{RESET}")
    print("-"*50)

Assistant:
Hello! I'm ARTIM, RMIT's AI Support Chatbot. I'm here to help you with any questions or issues you might have regarding Canvas, RMIT's Learning Management System. 

How can I assist you today?
--------------------------------------------------
You: How do I create a course discussion and how to view my assignments?
--------------------------------------------------
Fetching Necessary Info...
**************************************************
Fetching Necessary Info...
**************************************************
Assistant:
Based on the information retrieved, let me provide you with a concise response for both queries:

1. To create a course discussion:
- Go to Course Navigation and click "Discussions"
- Click "Add Discussion" button
- Enter a title and content using the Rich Content Editor
- You can set various options like:
  * Requiring participants to respond before viewing replies
  * Allowing likes on replies
  * Assigning to specific sections/students
  * Setting

In [16]:
conversation_history

[{'role': 'user', 'content': '👋'},
 {'role': 'assistant',
  'content': [{'type': 'text',
    'text': "Hello! I'm ARTIM, RMIT's AI Support Chatbot. I'm here to help you with any questions or issues you might have regarding Canvas, RMIT's Learning Management System. \n\nHow can I assist you today?"}]},
 {'role': 'user',
  'content': [{'type': 'text',
    'text': 'How do I create a course discussion and how to view my assignments?\nPlease break down my query into smaller sub-queries if necessary, BUT maintain **exact words** for each sub-query from the query above. Address each sub-query individually using the relevant tools. After obtaining information for each sub-query, synthesize the information to provide a consice response to my original query.'}]},
 {'role': 'assistant',
  'content': [{'type': 'text',
    'text': 'I\'ll break down your query into two sub-queries and search for information about each:\n\n1. "How do I create a course discussion"\n2. "how to view my assignments"\n\nLe

###### **Experiment 4** - Streaming with Tool calls 

In [ ]:
RED = "\033[31m"
GREEN = "\033[32m"
BLUE = "\033[34m"
RESET = "\033[0m"

In [83]:
credentials = get_credentials(USERNAME, PASSWORD)

In [84]:
system = """"
# ROLE DESCRIPTION
You are "ARTIM", an AI Support Chatbot at RMIT designed to assist students with queries/ troubleshooting related to Canvas Learning Management System.
---
# RESPONSIBILITIES
(VERY IMPORTANT) You are to always conduct yourself in a professional and ethical manner regardless of user's actions/ words.
(VERY IMPORTANT) Refrain from repling to off-topic questions, instead reply with a polite message stating your intended role/ responsibilities.
(VERY IMPORTANT) Strictly REFRAIN from generating harmful content
--
# RESPONSES
(VERY IMPORTANT) DO NOT SHARE thought process/ reasoning steps with the user.
(VERY IMPORTANT) Either directly provide the FINAL ANSWER to the user or use the 'fetch_canvas_guides' tool to obtain relevant information from Canvas guides to assist with the user's query.
---
# TOOLS - Sensitive Information DO NOT SHARE
(VERY IMPORTANT) If you have insufficient knowledge to assist with the user's query, always use 'fetch_canvas_guides' tool.
(VERY IMPORTANT) When creating a tool invocation for 'fetch_canvas_guides', NEVER modify the original user message. Pass the exact raw user text to the tool under "user_query".
(VERY IMPORTANT) For user queries with multiple questions in a single message, ALWAYS use parallel tool calls, each tool invocation taking one single question as input. After obtaining information for each sub-query, synthesize the information to provide a comprehensive response to the user's original query.
(VERY IMPORTANT) Once you have obtained information from 'fetch_canvas_guides' tool. Your answer to the user message is to be STRICTLY grounded in the obtained information. Do NOT use your general knowledge BUT use your common sense.
(VERY IMPORTANT) If you still find it hard to assist with user query despite consulting 'fetch_canvas_guides' tool, escalate the query to "RMIT IT SUPPORT".
---
## RMIT IT SUPPORT DETAILS
- Phone-support: +61399258000
- Email-support: support@rmit.com
"""


user_query_suffix = "\n---\nPlease break down my query into smaller sub-queries if necessary and make parallel tool calls."

In [85]:
bedrock = boto3.client(
    "bedrock-runtime",
    region_name=BEDROCK_REGION,
    aws_access_key_id=credentials["AccessKeyId"],
    aws_secret_access_key=credentials["SecretKey"],
    aws_session_token=credentials["SessionToken"],
)

body = json.dumps({
    "max_tokens": 1024, # required
    "temperature": 0,
    "anthropic_version": "bedrock-2023-05-31",
    "tools": [
      {
        "name": "fetch_canvas_guides",
        "description": "Based on user query, fetch related information from canvas guides",
        "input_schema": {
          "type": "object",
          "properties": {
              "user_query": {
                  "type": "string",
                  "description": "User query string *AS IS* to search related documents"
              }
          },
          "required": ["user_query"]
        }
      }
    ],
    "system": system,
    "messages": [{"role": "user", "content": "How to submit assignments on canvas, how to change canvas login password and how to check the rubric for my assignment"+ user_query_suffix}]
})



stream = bedrock.invoke_model_with_response_stream(
    body=body,
    modelId=MODEL_ID1,
)

In [18]:
stream

{'ResponseMetadata': {'RequestId': 'e2006d8a-f4fd-44c7-9102-008fe7645908',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 31 Oct 2025 09:28:09 GMT',
   'content-type': 'application/vnd.amazon.eventstream',
   'transfer-encoding': 'chunked',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'e2006d8a-f4fd-44c7-9102-008fe7645908',
   'x-amzn-bedrock-content-type': 'application/json'},
  'RetryAttempts': 0},
 'contentType': 'application/json',
 'body': <botocore.eventstream.EventStream at 0x10933c590>}

In [19]:
for sse in stream.get('body'):
    print(sse)

{'chunk': {'bytes': b'{"type":"message_start","message":{"id":"msg_bdrk_01FyKy4VFF8Qk8GG4t32n9Aa","type":"message","role":"assistant","model":"claude-3-5-sonnet-20241022","content":[],"stop_reason":null,"stop_sequence":null,"usage":{"input_tokens":887,"output_tokens":2}}}'}}
{'chunk': {'bytes': b'{"type":"content_block_start","index":0,"content_block":{"type":"text","text":""}}'}}
{'chunk': {'bytes': b'{"type":"content_block_delta","index":0,"delta":{"type":"text_delta","text":"I\'ll"}}'}}
{'chunk': {'bytes': b'{"type":"content_block_delta","index":0,"delta":{"type":"text_delta","text":" help"}}'}}
{'chunk': {'bytes': b'{"type":"content_block_delta","index":0,"delta":{"type":"text_delta","text":" you with your assignment"}}'}}
{'chunk': {'bytes': b'{"type":"content_block_delta","index":0,"delta":{"type":"text_delta","text":" submission"}}'}}
{'chunk': {'bytes': b'{"type":"content_block_delta","index":0,"delta":{"type":"text_delta","text":" issues"}}'}}
{'chunk': {'bytes': b'{"type":"co

In [86]:
tool_names = []
tool_ids = []
tool_inputs = []
input_str = ""

for sse in stream.get('body'):
    chunk = sse.get('chunk')
    event = json.loads(chunk.get('bytes'))
    
    if event.get('type') == 'content_block_start' and event.get('content_block').get('type') == 'tool_use':
        tool_name = event.get('content_block').get('name')
        tool_id = event.get('content_block').get('id')
        tool_names.append(tool_name)
        tool_ids.append(tool_id)
        continue
    
    
    if event.get('type') == 'content_block_delta' and event.get('delta').get('type') == 'text_delta':
        print(event.get('delta').get('text'), end='', flush=True)
        continue

    if event.get('type') == 'content_block_delta' and event.get('delta').get('type') == 'input_json_delta':
        input_str += event.get('delta').get('partial_json')
        continue
    
    if event.get('type') == 'content_block_stop' and input_str:
        tool_inputs.append(json.loads(input_str).get('user_query'))
        input_str = ""

I'll help you with your questions by breaking them down and searching for relevant information.

In [87]:
tool_names

['fetch_canvas_guides', 'fetch_canvas_guides', 'fetch_canvas_guides']

In [88]:
tool_ids

['toolu_bdrk_01DaEee6qma8xitJSiMmYMm3',
 'toolu_bdrk_01JHDSm6zVHpkYNyCBeUnUTa',
 'toolu_bdrk_015bfa7ZYYRWhsxUeUouZq8L']

In [89]:
tool_inputs

['How to submit assignments on canvas',
 'how to change canvas login password',
 'how to check assignment rubric in canvas']

##### 8 - ChatBedrockConverse LangChain

In [ ]:
! pip3 install langchain-aws

In [5]:
credentials = get_credentials(USERNAME, PASSWORD)
os.environ["AWS_ACCESS_KEY_ID"] = credentials["AccessKeyId"]
os.environ["AWS_SECRET_ACCESS_KEY"] = credentials["SecretKey"]
os.environ["AWS_SESSION_TOKEN"] = credentials["SessionToken"]

In [13]:
from langchain_aws import ChatBedrockConverse
from langchain_core.messages import AIMessageChunk

In [10]:
llm = ChatBedrockConverse(
    model_id=MODEL_ID1,
    region_name=BEDROCK_REGION,
)

In [13]:
messages = [
    ("human", "Hi"),
]

In [14]:
ai_msg = llm.invoke(messages)

In [15]:
ai_msg

AIMessage(content='Hello! How can I help you today?', additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': '530226eb-5b7b-4c25-98a4-c913c673fc78', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 01 Nov 2025 21:44:30 GMT', 'content-type': 'application/json', 'content-length': '233', 'connection': 'keep-alive', 'x-amzn-requestid': '530226eb-5b7b-4c25-98a4-c913c673fc78'}, 'RetryAttempts': 0}, 'stopReason': 'end_turn', 'metrics': {'latencyMs': [560]}, 'model_provider': 'bedrock_converse', 'model_name': 'anthropic.claude-3-5-sonnet-20241022-v2:0'}, id='lc_run--b4767c5d-09e0-40ee-80ea-0790e56017b9-0', usage_metadata={'input_tokens': 8, 'output_tokens': 12, 'total_tokens': 20, 'input_token_details': {'cache_creation': 0, 'cache_read': 0}})

In [20]:
for chunk in llm.stream(messages):
    print(chunk)

content=[] additional_kwargs={} response_metadata={'model_provider': 'bedrock_converse'} id='lc_run--7beb1574-f753-4540-9f92-999c8e6ad828'
content=[{'type': 'text', 'text': 'Hello! How', 'index': 0}] additional_kwargs={} response_metadata={'model_provider': 'bedrock_converse'} id='lc_run--7beb1574-f753-4540-9f92-999c8e6ad828'
content=[{'type': 'text', 'text': ' can', 'index': 0}] additional_kwargs={} response_metadata={'model_provider': 'bedrock_converse'} id='lc_run--7beb1574-f753-4540-9f92-999c8e6ad828'
content=[{'type': 'text', 'text': ' I help you today', 'index': 0}] additional_kwargs={} response_metadata={'model_provider': 'bedrock_converse'} id='lc_run--7beb1574-f753-4540-9f92-999c8e6ad828'
content=[{'type': 'text', 'text': '?', 'index': 0}] additional_kwargs={} response_metadata={'model_provider': 'bedrock_converse'} id='lc_run--7beb1574-f753-4540-9f92-999c8e6ad828'
content=[] additional_kwargs={} response_metadata={'model_provider': 'bedrock_converse'} id='lc_run--7beb1574-f75

In [23]:
for chunk in llm.stream(messages):
    print(chunk.text, end='', flush=True)

Hello! How can I help you today?

In [7]:
from langchain.tools import tool

In [8]:
import random

In [9]:
@tool
def guess_secret_number(secret_number: int) -> str:
    """Checks user's guess against the secret number."""
    SECRET = random.randint(1, 10)
    return f"{"You guessed it right!" if secret_number == SECRET else "Wrong guess, try again!"}"

In [36]:
guess_secret_number.invoke({"secret_number":7})

'You guessed it right!'

In [37]:
guess_secret_number.name

'guess_secret_number'

In [38]:
guess_secret_number.description

"Checks user's guess against the secret number."

In [39]:
guess_secret_number.args

{'secret_number': {'title': 'Secret Number', 'type': 'integer'}}

In [40]:
guess_secret_number.args_schema.model_json_schema() # LLM doesn't see the tool or the logic within in the function, it sees the tool's schema

{'description': "Checks user's guess against the secret number.",
 'properties': {'secret_number': {'title': 'Secret Number',
   'type': 'integer'}},
 'required': ['secret_number'],
 'title': 'guess_secret_number',
 'type': 'object'}

In [10]:
llm_with_tool = ChatBedrockConverse(
    model_id=MODEL_ID1,
    region_name=BEDROCK_REGION,
).bind_tools([guess_secret_number])

In [45]:
response = llm_with_tool.invoke('My guess is 7. Did I get it right?')

In [46]:
response

AIMessage(content=[{'type': 'text', 'text': "I'll help you check if 7 is the correct secret number."}, {'type': 'tool_use', 'name': 'guess_secret_number', 'input': {'secret_number': 7}, 'id': 'tooluse_EEcqTPaYQxKB8BGlOfaDEA'}], additional_kwargs={}, response_metadata={'ResponseMetadata': {'RequestId': 'c2b998e1-4c74-406c-87a8-e5f59722db64', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sat, 01 Nov 2025 22:04:15 GMT', 'content-type': 'application/json', 'content-length': '393', 'connection': 'keep-alive', 'x-amzn-requestid': 'c2b998e1-4c74-406c-87a8-e5f59722db64'}, 'RetryAttempts': 0}, 'stopReason': 'tool_use', 'metrics': {'latencyMs': [1746]}, 'model_provider': 'bedrock_converse', 'model_name': 'anthropic.claude-3-5-sonnet-20241022-v2:0'}, id='lc_run--df3d1601-2c61-4e39-b5cd-7410e6894e20-0', tool_calls=[{'name': 'guess_secret_number', 'args': {'secret_number': 7}, 'id': 'tooluse_EEcqTPaYQxKB8BGlOfaDEA', 'type': 'tool_call'}], usage_metadata={'input_tokens': 395, 'output_tokens': 73, 

In [47]:
response.tool_calls

[{'name': 'guess_secret_number',
  'args': {'secret_number': 7},
  'id': 'tooluse_EEcqTPaYQxKB8BGlOfaDEA',
  'type': 'tool_call'}]

In [ ]:
guess_secret_number.invoke(response.tool_calls[0]['args']) # sending only the arguments

'You guessed it right!'

In [57]:
guess_secret_number.invoke(response.tool_calls[0]) # sending only the arguments

ToolMessage(content='You guessed it right!', name='guess_secret_number', tool_call_id='tooluse_EEcqTPaYQxKB8BGlOfaDEA')

In [ ]:
for chunk in llm_with_tool.stream("My guess is 5. Did I get it right?"):
    print(chunk)

content=[] additional_kwargs={} response_metadata={'model_provider': 'bedrock_converse'} id='lc_run--edffe65c-e3fc-4d68-bbe9-896f05be1a35'
content=[{'type': 'text', 'text': 'I', 'index': 0}] additional_kwargs={} response_metadata={'model_provider': 'bedrock_converse'} id='lc_run--edffe65c-e3fc-4d68-bbe9-896f05be1a35'
content=[{'type': 'text', 'text': "'ll", 'index': 0}] additional_kwargs={} response_metadata={'model_provider': 'bedrock_converse'} id='lc_run--edffe65c-e3fc-4d68-bbe9-896f05be1a35'
content=[{'type': 'text', 'text': ' help', 'index': 0}] additional_kwargs={} response_metadata={'model_provider': 'bedrock_converse'} id='lc_run--edffe65c-e3fc-4d68-bbe9-896f05be1a35'
content=[{'type': 'text', 'text': ' you check', 'index': 0}] additional_kwargs={} response_metadata={'model_provider': 'bedrock_converse'} id='lc_run--edffe65c-e3fc-4d68-bbe9-896f05be1a35'
content=[{'type': 'text', 'text': ' if', 'index': 0}] additional_kwargs={} response_metadata={'model_provider': 'bedrock_conve

In [44]:
ai_msg = AIMessageChunk(content=[])
for chunk in llm_with_tool.stream("My guess is 5. Did I get it right?"):
    # if isinstance(chunk, AIMessageChunk):
    if chunk.content and chunk.content[0].get('type') == "text":
        print(chunk.text, end='', flush=True)
    ai_msg += chunk

I'll help you check if 5 is the secret number using the guess_secret_number function.

In [48]:
ai_msg

AIMessageChunk(content=[{'type': 'text', 'text': "I'll help you check if 5 is the secret number using the guess_secret_number function.", 'index': 0}, {'type': 'tool_use', 'name': 'guess_secret_number', 'id': 'tooluse_QQnV-th9Sc2sie4R1mvONg', 'index': 1, 'input': '{"secret_number": 5}'}], additional_kwargs={}, response_metadata={'model_provider': 'bedrock_converse', 'stopReason': 'tool_use', 'metrics': {'latencyMs': 1693}, 'model_name': 'anthropic.claude-3-5-sonnet-20241022-v2:0', 'ResponseMetadata': {'RequestId': '9a534a5b-99e8-45fa-8198-63e1094d2c49', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sun, 02 Nov 2025 00:16:04 GMT', 'content-type': 'application/vnd.amazon.eventstream', 'transfer-encoding': 'chunked', 'connection': 'keep-alive', 'x-amzn-requestid': '9a534a5b-99e8-45fa-8198-63e1094d2c49'}, 'RetryAttempts': 0}}, id='lc_run--60bf09c7-9051-40da-9cc0-b6f21318bfd0', tool_calls=[{'name': 'guess_secret_number', 'args': {'secret_number': 5}, 'id': 'tooluse_QQnV-th9Sc2sie4R1mvONg'

In [22]:
for chunk in llm_with_tool.stream("My guess is 5. Did I get it right?"):
   print(chunk.type)

AIMessageChunk
AIMessageChunk
AIMessageChunk
AIMessageChunk
AIMessageChunk
AIMessageChunk
AIMessageChunk
AIMessageChunk
AIMessageChunk
AIMessageChunk
AIMessageChunk
AIMessageChunk
AIMessageChunk
AIMessageChunk
AIMessageChunk
AIMessageChunk
AIMessageChunk
AIMessageChunk
AIMessageChunk
AIMessageChunk
AIMessageChunk
AIMessageChunk
AIMessageChunk
